## 0. 모듈 불러오기

In [1]:
!pip install pickle5

     |████████████████████████████████| 256 kB 5.0 MB/s 


In [2]:
import os
import glob
import json
import numpy as np
import pandas as pd
import time
from tqdm import trange, notebook
import pickle5 as pickle # colab에서 사용
# import pickle # 주피터에서 사용

### 구글 드라이브 마운트 및 디렉토리 변경


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/Othercomputers/내 노트북/07_PJT_final_01/02_style_recommender_system

/content/drive/Othercomputers/내 노트북/07_PJT_final_01/02_style_recommender_system


## 1. 데이터셋 가져오기

In [68]:
with open('./final_data.pkl', "rb") as fh:
  data = pickle.load(fh)

## 2. 함수 정의

In [52]:
# 선택한 상황을 반영하여 데이터셋 반환하는 함수
def select_condition(data, select_situation):
    if select_situation == 1:
        condition = (data['situation'] == '공')
    elif select_situation == 2:
        condition = (data['situation'] == '사')
    elif select_situation == 3:
        condition = (data['situation'] == '운동')
    elif select_situation == 4:
        condition = (data['situation'] == '기타')
    else:
        print('1 - 4 번 상황 중 선택해주세요')
        return None
    
    return data[condition]

In [53]:
# 온도에 따라 fabric, cloth_cat, arm_length 조건 반영하여 데이터셋 반환하는 함수
def select_temperature(data, temperature):
    if temperature < 5:
        fabric_list = ['가죽', '울캐시미어', '니트', '퍼', '패딩', '벨벳', '코듀로이','플리스', '무스탕', '자카드', '데님','스웨이드','헤어']
        cloth_cat_list = ['상의', '하의','아우터', '원피스']
        arm_length_list = ['긴팔',None]    
    elif temperature in range(5,9):
        fabric_list = ['데님', '니트', '스웨이드', '플리스', '트위드','울캐시미어','퍼', '벨벳', '스웨이드','코듀로이','헤어']
        cloth_cat_list = ['상의', '하의','아우터', '원피스']
        arm_length_list = ['긴팔',None]
    elif temperature in range(9,12):
        fabric_list = ['우븐', '가죽', '데님', '니트', '벨벳', '스웨이드','코듀로이','트위드','헤어']
        cloth_cat_list = ['상의', '하의','아우터', '원피스']
        arm_length_list = ['긴팔',None]
    elif temperature in range(12,17):
        fabric_list = ['우븐', '데님', '시폰', '레이스', '실크', '니트','시폰']
        cloth_cat_list = ['상의', '하의','아우터', '원피스']
        arm_length_list = ['긴팔',None]
    elif temperature in range(17,20):
        fabric_list = ['우븐', '가죽', '데님', '네오프렌','실크','시폰']
        cloth_cat_list = ['상의', '하의','아우터', '원피스']
        arm_length_list = ['7부소매','긴팔','반팔',None]
    elif temperature in range(20,23):
        fabric_list = ['린넨', '데님','시폰']
        cloth_cat_list = ['상의', '하의','아우터', '원피스']
        arm_length_list = ['7부소매','긴팔','반팔',None]
    elif temperature in range(23,28):
        fabric_list = ['린넨', '데님','메시','스판덱스']
        cloth_cat_list = ['상의', '하의', '원피스']
        arm_length_list = ['민소매','반팔','캡',None]
    elif temperature >= 28:
        fabric_list = ['린넨', '데님','메시','스판덱스']
        cloth_cat_list = ['상의', '하의', '원피스']
        arm_length_list = ['민소매','반팔','캡',None]
    else:
        print('잘못된 온도를 입력하셨습니다. 다시 입력해주세요.')
        return None

    data = data[data['fabric'].map(lambda x: any(x in select_fabric for select_fabric in fabric_list))]
    return data[data['cloth_cat'].isin(cloth_cat_list) & data['arm_length'].isin(arm_length_list)]

In [54]:
# 상황에 따라 스타일 키워드 리스트를 반환하는 함수
def return_style_keyword(select_situation):
    # 스타일 키워드 리스트
    public_list=['개성있는', '고전적', '남성적', '단순한', '도시적', '독특한', '모던함', '미래지향적', '산뜻함', '세련된', '여성스러운','우아함', '중성적', '지적', '편견없는', '편안한', '혁신적', '현대적인']
    other_list=['고급스러운', '고전적인', '다채로운', '레트로', '복고풍', '세련된', '슬림한', '자유분방한', '장식이과한', '화려한']
    privacy_list=['귀여운', '내추럴한', '단정한', '도시적', '독특한', '러블리한', '반항적인', '부드러운', '사랑스러운', '세련된', '소녀적인', '소박한', '야성적', '여성스러운', '자유분방한', '차분한', '캐주얼한', '편한', '현대적인', '힙한']

    if select_situation == 1: # public
        return public_list
    elif select_situation == 2:   # privacy
        return privacy_list
    elif select_situation == 4:   # other
        return other_list

In [55]:
# 상황에 따라 스타일 키워드 리스트를 반환하는 함수
def return_situation_style_keyword(select_situation):
    # 상황별 스타일 키워드 딕셔너리
    public_style = dict(매니시=['남성적', '중성적', '우아함', '산뜻함'],
                        모던=['도시적', '미래지향적', '모던함'],
                        소피스트케이티드=['세련된', '도시적', '지적', '여성스러운'],
                        아방가르드=['독특한', '혁신적'],
                        젠더리스=['중성적인', '개성있는', '편견없는'],
                        클래식=['고전적', '세련된', '우아함'],
                        톰보이=['남성적', '편안한', '단순한', '현대적인'])

    other_style = dict(레트로=['복고풍', '레트로'],
                    오리엔탈=['화려한', '고급스러운', '다채로운', '고전적인'],
                    키치=['장식이과한',  '자유분방한'],
                    히피=['자유분방한', '세련된', '화려한', '슬림한'])

    privacy_style = dict(로맨틱=['여성스러운', '귀여운', '사랑스러운', '소녀적인'],
                        스트리트=['현대적인', '자유분방한', '힙한'],
                        웨스턴=['도시적', '세련된', '야성적'],
                        컨트리=['내추럴한', '편한', '소박한', '소녀적인'],
                        펑크=['반항적인', '자유분방한', '독특한'],
                        페미닌=['여성스러운', '부드러운', '러블리한', '차분한'],
                        프레피=['캐주얼한', '단정한', '세련된'])

    if select_situation == 1: # public
        return public_style
    elif select_situation == 2:   # privacy
        return privacy_style
    elif select_situation == 4:   # other
        return other_style

In [69]:
# 상황별 스타일 키워드 선택에 따라 점수(가중치)을 반영한 데이터셋 반환하는 함수
def select_style(data, select_style_list, select_situation):
    # 선택한 상황에 따른 스타일 전체 키워드 리스트 생성
    style_keyword_list = return_style_keyword(select_situation)

    # 선택한 상황에 따른 스타일 키워드 딕셔너리 생성
    situation_style = dict(return_situation_style_keyword(select_situation))

    # 선택한 스타일 키워드에 따른 매트릭스 생성
    select_style_list = [int(i) for i in select_style_list.split(' ')]
    style_select_metrix = pd.DataFrame(np.zeros(shape=(len(data.fileID.unique()), len(style_keyword_list))))
    style_select_metrix['fileID'] = data.fileID.unique()

    # 선택한 키워드 : 1 / 미선택 키워드 : 0 반영
    for select in select_style_list:
        style_list = [k for k, v in situation_style.items() if style_keyword_list[select] in v]
        fname_list = data.loc[data['style'].isin(style_list), 'fileID'].unique()
        style_select_metrix.loc[style_select_metrix.fileID.isin(fname_list), select] += 1.

    # 스타일링 파일마다 선택된 키워드들의 합산한 값만 저장
    style_select_metrix['score'] = style_select_metrix.loc[:, select_style_list].sum(axis=1)
    style_select_metrix = style_select_metrix.loc[:, ['fileID','score']]

    # 키워드 점수를 원본데이터와 merge
    data = pd.merge(left = data , right = style_select_metrix, how = "inner", on = "fileID")
    
    # 최종적으로 사용할 컬럼만 반영하여 반환
    return data.loc[:, ['fileID', 'situation', 'style', 'cloth_cat', 'color', 'score', 'clothID']]

In [57]:
# 사용자 선호도 반영한 데이터셋 반환하는 함수
def create_user_faver(data, user_data):
    for user_cloth, user_color, user_score in zip(user_data.cloth_cat, user_data.color, user_data.favor):
        data.loc[((data.cloth_cat==user_cloth) & (data.color==user_color)),'score'] += user_score
    return data

In [58]:
def pre_processing_adjmatrix(data):
    # fileID별 중복이름(cloth_cat 다름) 제거를 위해 index 재설정
    df_name = pd.DataFrame(data.fileID.unique(), columns=['fileID'])
    df_name.reset_index(inplace=True)
    data = pd.merge(left = data , right = df_name, how = "inner", on = "fileID")
    
    # array 위치값으로 사용하기 위한 clothID index 재설정
    i = 0
    for id in data.clothID.unique():
        data.loc[data.clothID==id, 'clothID'] = i
        i+=1
    
    return data.loc[:, ['fileID', 'situation', 'style', 'cloth_cat', 'color', 'score', 'index', 'clothID']]

In [59]:
# 사용자 데이터 중 선택된 온도에 따라 데이터셋 반환하기
def return_user_temp(user_data, temperature):
    if temperature < 10:
        return user_data[user_data.season == '겨울']
    elif temperature in range(10,24):
        return user_data[(user_data.season == '봄')|(user_data.season == '가을')]
    elif temperature > 24:
        return user_data[user_data.season == '여름']

In [60]:
# 인접행렬 생성하기
def make_adj_matrix(data):
    data_array = np.array([[int(fileid), int(clothID), int(score), int(fname)] for fileid, clothID, score, fname in zip(data.index, data.clothID, data.score, data.fileID)])
    
    data_array[:,0] -= 1 # 0부터 시작하기 위해 1를 뺌
    fileid=int(max(data_array[:, 0])) # fileID 총 갯수
    clothID= int(max(data_array[:, 1]))#  clothID의 총 갯수
    shape = (clothID+1, fileid+1) # 전체 매트릭스 크기 확인

    adj_matrix=np.zeros(shape, dtype=int)
    for fileid, clothid, score, fname in data_array:
        adj_matrix[int(clothid)][int(fileid)] = int(score)
    
    return adj_matrix

In [61]:
# 코사인 유사도 구하기
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot/(norm1*norm2)

In [91]:
# 코사인 유사도를 계산하여 최종 TOP10 리스트 반환
# 거리가 가까울 수록(값이 작을 수록) 선택한 스타일 조건과 유사한 사용자

def make_recommend_list(adj_matrix, select_data):
    my_id, my_vector=0, adj_matrix[0]
    best_match, best_match_id, best_match_vector = 9999, -1, []

    for clothid, fileid in enumerate(adj_matrix):
        if my_id != clothid:
            cos_similarity = compute_cos_similarity(my_vector, fileid)
            if cos_similarity < best_match:
                best_match=cos_similarity
                best_match_id=clothid
                best_match_vector=fileid
    #print('best_match:',best_match, ',best_match_id:',best_match_id)

    recommend_list = []
    for i, log in enumerate(zip(my_vector, best_match_vector)):
        log1, log2 = log
        if log1 < 1. and log2 > 0.:
            recommend_list.append(i)

    return select_data[(select_data['index'].isin(list(recommend_list)))].sort_values('score', ascending=False).drop_duplicates('clothID')[:10]

In [63]:
# 최종데이터와 유저데이터 비교하여 옷장안의 유무 리스트(파일명) 반환
def return_match_user_data(final_data, user_data):
    match_list = []
    match_fail_list = []

    for fcat, fcolor, fname in zip(final_data.cloth_cat, final_data.color, final_data.fileID):
        if user_data[(user_data.cloth_cat==fcat)&(user_data.color==fcolor)].empty==False:
            match_list.append(fname)
        else:
            match_fail_list.append(fname)

    return match_list, match_fail_list

## 3. 실행하기

### 상황, 온도, 스타일 키워드 선택에 따른 데이터셋 생성

In [92]:
# 상황 선택하기
select_situation = int(input('1. 공적인 일\n2. 사적인 일\n3. 운동\n4. 기타(여행과 같은 이벤트성 상황)\n\n위 항목 중 필요한 상황을 번호로 선택해주세요.: '))
select_data = select_condition(data, select_situation)

1. 공적인 일
2. 사적인 일
3. 운동
4. 기타(여행과 같은 이벤트성 상황)

위 항목 중 필요한 상황을 번호로 선택해주세요.: 1


In [93]:
# 온도 선택하기
temperature = int(input('오늘의 온도를 입력하세요.: '))
select_data = select_temperature(select_data, temperature)

오늘의 온도를 입력하세요.: 12


In [94]:
# 앞서 선택한 상황에 대한 스타일 키워드 리스트 반환하기 1 2 3 6 9 11 12 13
select_style_list = return_style_keyword(select_situation)
select_style_list = input(f'{select_style_list}\n원하는 스타일을 골라주세요.\n')

['개성있는', '고전적', '남성적', '단순한', '도시적', '독특한', '모던함', '미래지향적', '산뜻함', '세련된', '여성스러운', '우아함', '중성적', '지적', '편견없는', '편안한', '혁신적', '현대적인']
원하는 스타일을 골라주세요.
1 2 3 6 9 11 12 13


In [95]:
# 스타일 키워드 선택하기
select_data = select_style(select_data, select_style_list, select_situation)

### n개의 랜덤한 테스트 데이터 생성

In [96]:
# 입력날짜 컬럼 필요

In [97]:
n = 20
season = ['봄', '여름', '가을', '겨울']
color = {'골드': 20,
        '그레이': 8,
        '그린': 14,
        '네온': 21,
        '네이비': 7,
        '라벤더': 17,
        '레드': 11,
        '민트': 19,
        '베이지': 3,
        '브라운': 9,
        '블랙': 2,
        '블루': 4,
        '스카이블루': 5,
        '실버': 12,
        '옐로우': 13,
        '오렌지': 16,
        '와인': 15,
        '카키': 10,
        '퍼플': 18,
        '핑크': 6,
        '화이트': 1}
cat = {'상의': 1, '아우터': 3, '원피스': 4, '하의': 2}

In [98]:
import random
cat_num = [random.randint(1, 4) for x in range(n)]
color_num = [random.randint(1, 21) for x in range(n)]
season_num = [random.randint(0, 3) for x in range(n)]
favor_num = [random.randint(0, 4) for x in range(n)]

In [99]:
user_data = pd.DataFrame([[f'cloth{i}', cat_num[i], color_num[i], season[season_num[i]], favor_num[i]] for i in range(0,n,1)], columns=['user_cloth_no', 'cloth_cat', 'color', 'season', 'favor'])

In [100]:
for k, v in color.items(): user_data.loc[user_data.color==v, 'color'] = k
for k, v in cat.items(): user_data.loc[user_data.cloth_cat==v, 'cloth_cat'] = k

In [101]:
user_data = return_user_temp(user_data, temperature)

In [102]:
user_data

,user_cloth_no,cloth_cat,color,season,favor
0,cloth0,원피스,핑크,가을,1
2,cloth2,하의,카키,봄,4
9,cloth9,원피스,골드,봄,4
11,cloth11,원피스,블루,봄,4
13,cloth13,아우터,골드,가을,1
14,cloth14,하의,블루,봄,4
16,cloth16,하의,네이비,봄,4
17,cloth17,원피스,레드,봄,0
19,cloth19,아우터,민트,가을,4


### 추천해주기

In [103]:
ts = time.time()
select_data = create_user_faver(select_data, user_data)
select_data = pre_processing_adjmatrix(select_data)
adj_matrix = make_adj_matrix(select_data)
print(f'소요시간: {time.time()-ts}s')

소요시간: 5.00740385055542s


In [104]:
recommend_data = make_recommend_list(adj_matrix, select_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


best_match: 0.0 ,best_match_id: 1


In [112]:
recommend_data

,fileID,situation,style,cloth_cat,color,score,index,clothID
2976,225774,공,매니시,하의,블루,7.0,2287,15
4770,421639,공,매니시,하의,네이비,7.0,3834,47
2728,184922,공,매니시,하의,카키,7.0,2055,44
1207,1181232,공,매니시,하의,네이비,7.0,910,64
1203,1180919,공,매니시,하의,블루,7.0,906,10
11197,1048411,공,모던,하의,네이비,5.0,8504,197
6948,797036,공,매니시,아우터,그레이,3.0,5415,5
0,1002940,공,매니시,하의,와인,3.0,0,0
6635,747381,공,매니시,하의,베이지,3.0,5211,8
6636,747381,공,매니시,상의,블랙,3.0,5211,14


#### 벡터내적과 유클리디안 거리로 유사도 확인하기

In [105]:
# 벡터내적
my_id, my_vector=0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match=similarity
      best_match_id=user_id
      best_match_vector=user_vector
print('best_match:',best_match, ',best_match_id:',best_match_id)

best_match: 6 ,best_match_id: 1269


In [106]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[153352, 153354, 153356, 153358, 153359, 153360, 153361, 153363, 153365, 153367, 153369, 153371, 153373, 153375, 153377, 153379, 153381, 153383, 153385, 153386, 153388, 153390, 153392, 153394, 153531, 153532, 153533, 153534, 153535, 153536, 153537, 153538, 153539, 153540, 153542, 153544, 153545, 153546, 153547, 153549, 153570, 153571, 153572, 153586, 153589, 153590, 153593, 153594, 153595, 153596, 153597, 153598, 153710, 153714, 153873, 153878, 153879, 153887, 153889, 153911, 153912, 153913, 153914, 153922, 154227, 154229, 154231, 154233, 154235, 154237, 154238, 154239, 154241, 154243, 154245, 154247, 154249, 154251, 154253, 154255, 154257, 154259, 154261, 154263, 154264, 154266, 154268, 154270, 154272, 154274, 154276, 154278, 154280, 154282, 154284, 154286, 154288, 154290, 154292, 154294, 154296, 154298, 154300, 154302, 154304, 154305, 154306, 154307, 154308, 154309, 154310, 154311, 154312, 154313, 154314, 154315, 154316, 154317, 154318, 154319, 154320, 154321, 154322, 154323, 154324,

In [107]:
# 유클리디안 
my_id, my_vector=0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match=euclidean_dist
      best_match_id=user_id
      best_match_vector=user_vector

print('best_match:',best_match, ',best_match_id:',best_match_id)

best_match: 30.44667469527666 ,best_match_id: 717


In [108]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[]


### 추천리스트와 사용자 비교하여 보여주기


In [109]:
match_list, match_fail_list = return_match_user_data(recommend_data, user_data)

In [110]:
data[data.fileID.isin(match_list)]

,fileID,situation,style,cloth_cat,color,pattern,length,arm_length,fit_cloth,fabric,clothID
2209,1180919,공,매니시,하의,블루,무지,발목,None,스키니,데님,10243
2210,1180919,공,매니시,상의,네이비,스트라이프,None,7부소매,노멀,,10170
2219,1181232,공,매니시,하의,네이비,,발목,None,루즈,우븐,10271
2220,1181232,공,매니시,상의,화이트,,None,반팔,노멀,우븐,10111
4593,184922,공,매니시,하의,카키,무지,발목,None,노멀,우븐,102101
5003,225774,공,매니시,하의,블루,무지,발목,None,노멀,우븐,10241
5004,225774,공,매니시,상의,화이트,,None,민소매,None,,10110
7676,421639,공,매니시,아우터,그레이,체크,롱,긴팔,루즈,우븐,10381
7677,421639,공,매니시,하의,네이비,,발목,None,스키니,데님,10273
17171,1048411,공,모던,하의,네이비,무지,발목,None,노멀,우븐,3271


In [111]:
data[data.fileID.isin(match_fail_list)]

,fileID,situation,style,cloth_cat,color,pattern,length,arm_length,fit_cloth,fabric,clothID
0,1002940,공,매니시,하의,와인,무지,발목,None,와이드,우븐,102151
1,1002940,공,매니시,상의,화이트,무지,None,긴팔,노멀,우븐,10111
10177,747381,공,매니시,하의,베이지,무지,발목,None,와이드,우븐,10231
10178,747381,공,매니시,상의,블랙,무지,노멀,긴팔,루즈,니트,10124
10563,797036,공,매니시,아우터,그레이,체크,하프,긴팔,루즈,우븐,10381
